# Adatbázisok

Ha adatkezelésről beszélünk, nem mehetünk el szó nélkül a jó öreg relációs adatbáziskezelők mellett. Ma már számtalan "modern" forma létezik, mégis az fontos adataink igen jelentős részét ezek a bevált rendszerek őrzik.

A python "klasszikus" adatbázis kezelő absztrakciója az SQLAlchemy, amely egy úgynevezett ORM (object-relational mapper) azaz a relációs adatbázist automatikusan-félautomatikusan objektumokká képezi le.

Mivel ez elég "programozó" dolog, és igazából nem is túl hasznos, ha nem akarunk hatalmas SQL motor független szoftverrendszereket évekig karbantartani, itt most egy sokkal egyszerűbb lehetőséget mutatok be: az SQLite motorhoz való python csomagot amely direkt SQL nyelvi utasításokat kezel.

Ha még soha nem tanultál SQL-t akkor valószínűleg nem lesz teljesen világos, miért olyan nagyon hasznos ha ilyen egyszerű adatbázisokat létrehozni és kezelni, de ha ismered az SQL nyelvet, akkor egy rendkívül jó adatkezelő eszközhöz jutsz lényegében ingyen.

Az SQLite egy szuper könnyűsúlyú, szerver nélküli adatbázis motor, ami minden telepítés nélkül, lokálisan elérhetővé teszi neki nekünk a relációs adatbázisok számtalan előnyös tulajdonságát.

In [ ]:
# Mit már megszokhattuk egy importtal indítunk
import sqlite3

Mivel az SQLite esetében nincs szerver, az sqlite "kapcsolat" lényegében annyit jelent, hogy megnyitunk egy fájlt, aminek megadjuk a nevét. Egy "rendes" adatbázis kezelőnél, authentikációs adatokat, szervernevet, portot és schema vagy adatbázis nevet kellene megadnunk, itt mindez nem szükséges.  

In [ ]:
# megnyitjuk a "kapcsolatot"
con = sqlite3.connect("minta.db")

Ezen a Connection objektumon keresztül fogjuk tudni az adatbázist kezelni. Természetesen, akárcsak a fájloknál, ezt is illene a végén lezárnunk! Ha nem tesszük, kockáztatjuk, hogy a módosításaink eltűnnek.

Ha valamit végre szeretnénk hajtani, kérnünk ekll egy adatbázis kurzort, amivel aztán végrehajthatjuk az SQL parancsokat. Ilyen kurzorunk egyszerre több is lehet.

In [ ]:
cur = con.cursor()
# és máris futtathatunk bármilyen SQL-t.

cur.execute("CREATE TABLE adatok(name TEXT, age INT)")

Kész, a táblánk máris létrejött. Az adatokat beletehetnénk egyszerű INSERT utasításokkal, de az adat érvényesítés miatt ez nem tanácsos (nehéz ellenőrizni és jól escapelni az adatokat), ezért inkább "placeholder" technikát használjuk.
Az adatok a kérdőjelek helyére kerülnek.

Az insert utasítás (minden módosítás) automatikusan tranzakciót indít az SQLite motorban, így ha a módosítást befejeztük, az SQL-ben megszokott módon commit utasítással le kell zárnunk a műveletsort. (Avagy ROLLBACK utasítással elvetjük, ha valami félrement).

In [ ]:
adatok = [
    ("Péter", 23),
    ("Eszter", 19),
    ("Kinga", 33),
]

cur.executemany("INSERT INTO adatok VALUES(?, ?)", adatok)
con.commit() # befejezzük a tranzakció (kiírjuk a módosítást)

In [ ]:
# Kérjük vissz az adatokat!
for row in cur.execute("SELECT * FROM adatok ORDER BY name"):
    print(row)

('Eszter', 19)
('Kinga', 33)
('Péter', 23)


In [ ]:
# Végül, (ha már nem használjuk) zárjuk le az adatbázisunkat:
con.close()

## Memória adatbázis

Az SQLite működéséhez igazából még fájl sem kell. Boldogan elfut tisztán a memóriában is, tehát ha nincs szükségünk állandó adatra, futtathatjuk ideiglenesen a memóriából is.

Ezúttal akkor használjunk elegáns környezetkezelőt, a fapados kézi lezárás helyett!

In [ ]:
con = sqlite3.connect(":memory:")

# a Connection-el is végrehajthatunk SQL-t magától csinál egy Cursort
cur = con.execute("CREATE TABLE test(cat, val)")

values = [ # tárolandó minta adat
    ("a", 4),
    ("b", 5),
    ("b", 3),
    ("a", 8),
    ("a", 1),
]

with con: # automatikus commit
  cur.executemany("INSERT INTO test VALUES(?, ?)", values)

# és kérdezzünk le belőle valami érdekes statisztikát
cur.execute("""
  SELECT cat, sum(val) as sum, avg(val) as avg
  FROM test GROUP BY cat
""")

for cat, sum, avg in cur.fetchall():
  print(cat, sum, avg)

a 13 4.333333333333333
b 8 4.0


## Adatbázis gyakorlat

Nézzünk egy gyakorlati példát. Tegyük fel, hogy az adatainkat egy könyvtár hierarchiában tároljuk és gyakran van szükségünk onnan adatokra. A fájlrendszeren keresgélni viszont lassú (lehet, hogy például hálózati meghajtóról van szó), ezért úgy döntöttünk, hogy építünk belőle egy adatbázist, amiben sokkal gyorsabb lesz keresni!

Gyakorlásképp olvassuk a /usr/share könyvtár tartalmát adatbázisba! (Ott jó sok fájl van).

In [ ]:
from re import X
import sqlite3
from pathlib import Path

# az most csak a memoriában lesz meg, de lehetne fájl...
con = sqlite3.connect(":memory:")
# készítünk egy szuper egyszerű táblát az adatoknak
cur = con.execute("CREATE TABLE files(fn, size)")

with con:
  for p in Path('/usr/share').glob("**/*"):
    if not p.is_file():
      continue
    méret = p.stat().st_size # ez a fájlméret
    cur.execute("INSERT INTO files VALUES (?,?)", (p.name, méret))

# és máris szuper gyorsan tudunk bármit lekérdezni:

In [ ]:
# hány fájl van ami 3000 bájtnál hosszabb?
cur.execute('SELECT count(*) FROM files WHERE size>3000');
cur.fetchall()

In [ ]:
# legnagyobb fájlméret?
cur.execute('SELECT max(size) FROM files');
cur.fetchone()

In [ ]:
# melyik fáljnévből van több mint 30, darabszám szerint rendezve?
cur.execute('SELECT fn, count(*) db FROM files GROUP BY fn HAVING db>30 ORDER BY -db');
cur.fetchall()

Ennyi az egész! Persze a fájlméret helyett vagy mellett más adatot is tárolhattunk volna, például a dátumot vagy akár a fájl tartalmát is indexelhetjük (ezt azért ne itt most ne tedd, mert az /usr/share alatt nagyon sok fájl van). Inkább:

1. Alakítsd át a kódot, hogy az útvonalat is tárolja a fájlnév mellett, ha esetleg valaki pontosan tudni szeretné, hol vannak a fájlok!

2. Készíts egy `input` ciklust, ami a felhasználótól kér be egy név-részletet és visszaadja az összes olyan fájlt amiben szerepel (ha nem tanultál ilyet, SQL-ben ez így tudod megcsinálni: `WHERE fn LIKE '%részlet%'`, de nyugodtan kérdezz meg egy LLM modellt)